In [2]:
## Importing packages

import datetime
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.linear_model import LinearRegression

In [47]:
## Index and ticker

ticker = 'GGBR4.SA'
index = '^BVSP'

In [48]:
## Time frame

start_date = datetime.datetime(2018, 7, 9)
end_date = datetime.datetime(2022, 7, 9)


In [49]:
## Retireving ticker and index information

ticker_data = yf.Ticker(ticker)
index_data = yf.Ticker(index)

In [50]:
## GGBR4 daily returns

ticker_historical_price = ticker_data.history(start=start_date, end=end_date).reset_index()
ticker_historical_price = ticker_historical_price[['Date', 'Close']]
ticker_historical_price[ticker + '_daily_return'] = ticker_historical_price['Close'].pct_change(1)
ticker_historical_price = ticker_historical_price.dropna()
ticker_historical_price.head()


,Date,Close,GGBR4.SA_daily_return
1,2018-07-11,12.243563,-0.023606
2,2018-07-12,12.909600,0.054399
3,2018-07-13,12.753368,-0.012102
4,2018-07-16,12.934265,0.014184
5,2018-07-17,13.287841,0.027336


In [51]:
## Ibovespa daily returns

index_historical_price = index_data.history(start=start_date, end=end_date).reset_index()
index_historical_price = index_historical_price[['Date', 'Close']]
index_historical_price[index + '_daily_return'] = index_historical_price['Close'].pct_change(1)
index_historical_price = index_historical_price.dropna()
index_historical_price.head()


,Date,Close,^BVSP_daily_return
1,2018-07-11,74399.0,-0.006185
2,2018-07-12,75856.0,0.019584
3,2018-07-13,76594.0,0.009729
4,2018-07-16,76653.0,0.000770
5,2018-07-17,78130.0,0.019269


In [52]:
## Leveraged beta

y = np.array(ticker_historical_price['GGBR4.SA_daily_return'])
x = np.array(index_historical_price['^BVSP_daily_return']).reshape((-1, 1))

model = LinearRegression()
model.fit(x, y)

leveraged_beta = model.coef_
print(leveraged_beta)


[1.13721496]


In [53]:
## Unleveraged beta

debt_to_equity = ticker_data.info['debtToEquity']/100
tax_rate = 0.34
unleveraged_beta = leveraged_beta/(1+debt_to_equity*(1-tax_rate))
print(debt_to_equity, unleveraged_beta)


0.32222 [0.93778147]


In [6]:
## Risk-free tax

def busca_titulos_tesouro_direto():
  url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
  df = pd.read_csv(url, sep=';', decimal=',')
  df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
  df['Data Base'] = pd.to_datetime(df['Data Base'], dayfirst=True)
  multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
  df = df.set_index(multi_indice).iloc[:, 3:]
  return df


titulos = busca_titulos_tesouro_direto()
titulos.sort_index(inplace=True)

tesouro_pre_2025 = titulos.loc[('Tesouro Prefixado', '2025-01-01')].reset_index()
risk_free_tax = tesouro_pre_2025[tesouro_pre_2025['Data Base'] == tesouro_pre_2025['Data Base'].max()]['Taxa Compra Manha']/100
print(risk_free_tax)


1097    0.1285
Name: Taxa Compra Manha, dtype: float64


In [54]:
## Market return

premium = 0.05
market_return = risk_free_tax + premium


In [55]:
## Discount rate

discount_rate = risk_free_tax + leveraged_beta*(market_return-risk_free_tax)
print(discount_rate)

1097    0.185361
Name: Taxa Compra Manha, dtype: float64
